In [ ]:
import math
import random

import numpy as np

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Простая Синусоида</h3>

In [3]:
def generate_sinus(n_samples):
    epsilon = np.random.normal(size=(n_samples))
    x  = np.random.uniform(-10.5, 10.5, n_samples)
    y  = 7 * np.sin(0.75 * x) + 0.5 * x + epsilon
    return torch.FloatTensor(x), torch.FloatTensor(y)
    
n_samples = 1000
x, y = generate_sinus(n_samples)

x = x.unsqueeze(1).to(device)
y = y.unsqueeze(1).to(device)

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(x.squeeze().tolist(), y.squeeze().tolist(), alpha=0.2)
plt.show()

In [8]:
def plot(epoch, losses, x, y, y_pred):
    clear_output(True)
    plt.figure(figsize=(20,5))
    
    plt.subplot(132)
    plt.scatter(x, y, alpha=0.2)
    plt.scatter(x, y_pred, alpha=0.2)
    
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (epoch, losses[-1]))
    plt.plot(losses)
    
    plt.show()

<h3>Модель</h3>

In [11]:
#code here

In [ ]:


epoch = 0
num_epochs = 1000

losses = []

<h3>Вертикальная Синусоида</h3>

In [13]:
x, y = y, x

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(x.squeeze().tolist(), y.squeeze().tolist(), alpha=0.2)
plt.show()

In [ ]:
epoch = 0
num_epochs = 1000

losses = []

<h3>Gaussian Mixture Models</h3>

In [15]:
class GMM(nn.Module):
    def __init__(self, hidden_size, num_gaussians):
        super(GMM, self).__init__()
        
    def forward(self, x):
        '''
        Inputs:
            x: (batch x 1)
        Outputs:
            mean:   (batch x num_gaussians)
            std:    (batch x num_gaussians)
            weight: (batch x num_gaussians)
            
        '''
        #code here
    
    def sample(self, x):
        mean, std, weight = self.forward(x)
        
        gumbel  = np.random.gumbel(loc=0, scale=1, size=weight.size())
        gumbel  = torch.FloatTensor(gumbel).to(x.device)

        indices = (weight.log() + gumbel).max(1)[1]
        indices = indices.unsqueeze(1)

        mean = mean.gather(1, indices).squeeze(1)
        std  = std.gather(1,  indices).squeeze(1)

        sampled = torch.randn(n_samples).to(x.device) * std + mean
        return sampled

<h4>Normal Distribution</h4>

In [19]:
def pdf(y, mean, std):
    '''
    Inputs:
        y:    (batch x 1)
        mean: (batch x num_gaussians)
        std:  (batch x num_gaussians)
    Outputs:
        dist: (batch x num_gaussians)
    '''
    #
    #code here
    #

<h4>GMM Loss</h4>

In [20]:
def compute_gmm_loss(mean, std, gaussian_weight, y):
    '''
    Inputs:
        mean: (batch x num_gaussians)
        std:  (batch x num_gaussians)
        gaussian_weight: (batch x num_gaussians)
        y:    (batch x 1)
    '''
    #
    #code here
    #

In [ ]:
gmm = GMM(20, 20).to(device)
optimizer = optim.RMSprop(gmm.parameters())

epoch = 0
num_epochs = 1000

losses = []

In [ ]:
gaussian_weight, mean, std = gmm(x)

while epoch < num_epochs:
    
    mean, std, weight = gmm(x)
    loss = compute_gmm_loss(mean, std, weight, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 10 == 0:
        y_pred = gmm.sample(x)
        plot(epoch, losses, x.squeeze().tolist(), y.squeeze().tolist(), y_pred.squeeze().tolist())
    
    epoch += 1

<h2>Задание</h2>
<p>Попробуйте заменить tanh на relu и sigmoid и посмотрите что будет [1 бал]</p>
<p>Попробуйте использовать softplus вместо exp при расчете std и посмотрите что будет [1 бал]</p>
<p>Попробуйте использовать разное кол-во гауссиан и сделайте не большой отчет [1 бал]</p>